$$%
\newcommand{\BB}{{\boldsymbol B}}%
\newcommand{\cc}{{\boldsymbol c}}%
\let\dd\relax%
\newcommand{\dd}{{\boldsymbol d}}%
\newcommand{\DD}{{\boldsymbol D}}%
\newcommand{\Id}{{\boldsymbol I}}%
\newcommand{\HH}{{\boldsymbol H}}%
\newcommand{\RR}{{\boldsymbol R}}%
\newcommand{\TT}{{\boldsymbol T}}%
\newcommand{\VV}{{\boldsymbol V}}%
\newcommand{\WW}{{\boldsymbol W}}%
\newcommand{\ww}{{\boldsymbol w}}%
\newcommand{\XX}{{\boldsymbol X}}%
\renewcommand{\aa}{{\boldsymbol a}}%
\newcommand{\xx}{{\boldsymbol x}}%
\newcommand{\yy}{{\boldsymbol y}}%
% Math Symbols
\newcommand{\rrh}{{\boldsymbol \rho}}%
\newcommand{\mmu}{{\boldsymbol \mu}}%
\newcommand{\ssi}{{\boldsymbol \sigma}}%
\newcommand{\SSi}{{\boldsymbol \Sigma}}%
\newcommand{\eps}{{\boldsymbol \epsilon}}%
\newcommand{\GGa}{{\boldsymbol \Gamma}}%
\newcommand{\bz}{{\boldsymbol 0}}%
\DeclareMathOperator{\car}{CAR}%
\newcommand{\N}{\mathcal N}%
\newcommand{\iid}{\stackrel{iid}{\sim}}%
\newcommand{\ud}{\, \mathrm{d}}%
\newcommand{\tmin}{t_{\mathrm{min}}}%
\newcommand{\tmax}{t_{\mathrm{max}}}%
\newcommand{\carp}{\mathrm{CAR}(p)}
$$
## KalmanODE & KalmanODE_py

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from KalmanODE_py import KalmanODE_py
from probDE.car import car_init
from probDE.cython.KalmanODE import KalmanODE
from probDE.utils.utils import rand_mat, indep_init
from scipy.integrate import odeint

%load_ext cython

In [2]:
from math import sin
from scipy import integrate
def ode_py(x_t, t, theta=None):
    return sin(2*t) - x_t[0]

# ode function in odeint format
def f(x_t, t):
    return [x_t[1], sin(2*t) - x_t[0]]

In [29]:
# LHS vector of ODE
w_vec = np.array([0.0, 0.0, 1.0])

# These parameters define the order of the ODE and the CAR(p) process
n_meas = 1
n_state = 4

# it is assumed that the solution is sought on the interval [tmin, tmax].
n_eval = 100
tmin = 0
tmax = 10

# The rest of the parameters can be tuned according to ODE
# For this problem, we will use
tau = 50
sigma = .5

# Initial value, x0, for the IVP
x0 = np.array([-1., 0., 1.])

In [30]:
# Get parameters needed to run the solver
dt = (tmax-tmin)/n_eval
# All necessary parameters are in kinit, namely, T, c, R, W
kinit, x0_state = indep_init([car_init(n_state, tau, sigma, dt, w_vec, x0)], n_state)

# Initialize the Kalman class
kalmanode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, ode_py, **kinit)
# Run the solver to get an approximation
kalman_sim = kalmanode.solve(x0_state, mv=False, sim=True)

In [31]:
# Timings for KalmanODE
kalmanode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, ode_py, **kinit) # Initialize the class
kalman_cy = %timeit -o -n 100 _ = kalmanode.solve(x0_state, mv=False, sim=True)

# Timings for KalmanODE in pure Python
kalmanode_py = KalmanODE_py(n_state, n_meas, tmin, tmax, n_eval, ode_py, **kinit) # Initialize the class
kalman_py = %timeit -o -n 100 _ = kalmanode_py.solve(x0_state, mv=False, sim=True)

# Timings for odeint
tseq = np.linspace(tmin, tmax, n_eval+1)
ode = %timeit -o ode = integrate.odeint(f, [-1, 0], tseq)
print('py/cy: {}'.format(kalman_py.average/kalman_cy.average))
print('ode/cy: {}'.format(ode.average/kalman_cy.average))

361 µs ± 15.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.6 ms ± 172 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
535 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
py/cy: 48.76090277217132
ode/cy: 1.4827028554332742


## Lorenz63

In [8]:
def lorenz0(X_t, t, theta=(28, 10, 8/3)):
    rho, sigma, beta = theta
    p = 3
    x, y, z = X_t[0], X_t[1], X_t[2]
    return np.array([-sigma*x + sigma*y, rho*x - y -x*z, -beta*z + x*y]) 

In [24]:
# RHS of ODE
def lorenz(X_t, t, theta=(28, 10, 8/3)):
    rho, sigma, beta = theta
    p = 2
    x, y, z = X_t[p*0], X_t[p*1], X_t[p*2]
    return np.array([-sigma*x + sigma*y, rho*x - y -x*z, -beta*z + x*y])

# LHS Matrix of ODE
w_mat = np.array([[0.0, 1.0], [0.0, 1.0], [0.0, 1.0]])

# These parameters define the order of the ODE and the CAR(p) process
n_meas = 3
n_state = 6 # number of continuous derivatives of CAR(p) solution prior
n_state1 = n_state2 = n_state3 = 2
n_var_states = np.array([n_state1, n_state2, n_state3])

# it is assumed that the solution is sought on the interval [tmin, tmax].
n_eval = 5000
tmin = 0
tmax = 20
theta=(28, 10, 8/3)

# The rest of the parameters can be tuned according to ODE
# For this problem, we will use
tau = np.array([1.3, 1.3, 1.3])
sigma = np.array([.5, .5, .5])

# Initial value, x0, for the IVP
x0 = [-12, -5, 38]
v0 = [70, 125, -124/3]
x0 = np.column_stack([x0, v0])

In [ ]:
# Get parameters needed to run the solver
dt = (tmax-tmin)/n_eval
# Initialize CAR with three variables
kinit, x0_state = indep_init([car_init(n_state1, tau[0], sigma[0], dt, w_mat[0], x0[0]),
                        car_init(n_state2, tau[1], sigma[1], dt, w_mat[1], x0[1]),
                        car_init(n_state3, tau[2], sigma[2], dt, w_mat[2], x0[2])], n_state)

# Initialize the Kalman class
kalmanode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, lorenz, **kinit)
# Run the solver to get an approximation
kalman_lor_sim = kalmanode.solve(x0_state, theta, mv=False, sim=True)

In [26]:
kalmanode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, lorenz, **kinit) # Initialize the class
kalman_lor_cy = %timeit -o _ = kalmanode.solve(x0_state, theta, mv=False, sim=True)
kalmanode_py = KalmanODE_py(n_state, n_meas, tmin, tmax, n_eval, lorenz, **kinit) # Initialize the class
kalman_lor_py = %timeit -o _ = kalmanode_py.solve(x0_state, theta, mv=False, sim=True)
print('py/cy: {}'.format(kalman_lor_py.average/kalman_lor_cy.average))

49.7 ms ± 1.66 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
982 ms ± 37.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
py/cy: 19.775811622851297


## Lane-Emden

In [32]:
def lane(x_t, t, theta=None):
    return np.array([-2/t*x_t[1] - x_t[0]**5])

# LHS vector of ODE
w_vec = np.array([0., 0., 1.])

# These parameters define the order of the ODE and the CAR(p) process
n_meas = 1
n_state = 3

# it is assumed that the solution is sought on the interval [tmin, tmax].
n_eval = 200
tmin = 0
tmax = 10

# The rest of the parameters can be tuned according to ODE
# For this problem, we will use
tau = 10
sigma = .001

# Initial value, x0, for the IVP
x0 = np.array([1., 0., -1.]) #Initial State

In [33]:
# Get parameters needed to run the solver
dt = (tmax-tmin)/n_eval
kinit, x0_state = indep_init([car_init(n_state, tau, sigma, dt, w_vec, x0)], n_state)

# Cython
kalmanode = KalmanODE(n_state, n_meas, tmin, tmax, n_eval, lane, **kinit) # Initialize the class
kalman_lane_sim = kalmanode.solve(x0_state, mv=False, sim=True)

In [34]:
kalman_lane_cy = %timeit -o _ = kalmanode.solve(x0_state, mv=True, sim=True)
kalmanode_py = KalmanODE_py(n_state, n_meas, tmin, tmax, n_eval, lane, **kinit) # Initialize the class
kalman_lane_py = %timeit -o _, _, _ = kalmanode_py.solve(x0_state, mv=True, sim=True)
print('py/cy: {}'.format(kalman_lane_py.average/kalman_lane_cy.average))

1.16 ms ± 16.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
48.5 ms ± 1.71 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
py/cy: 41.88374569933891
